# Data collecting

## a.Functions used for data scraping

In [22]:
%pip install bs4

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\HCMUS_Document\Năm 3\Kì 1\Machine Learning\Intro2ML-HanhTrinhChuViet\lab 1\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [23]:
import re
import requests
import os, csv, re, time, random
from bs4 import BeautifulSoup
from urllib.parse import urljoin,urlsplit, urlunsplit, parse_qsl, urlencode

s = requests.Session()
def get_soup(url, timeout=20):
    r = s.get(url, timeout=timeout)
    if r.status_code != 200:
        return None
    return BeautifulSoup(r.text, "html.parser") 

def clean_text(t):
    return re.sub(r"\s+", " ", (t or "")).strip()

def parse_price_ty_vnd(text):
    if not text: return None, None
    t = clean_text(text).lower()
    raw = t
    list = t.split(" ")
    total = 0
    for i in range(len(list)):
        if "tỷ" in list[i]:
            total = total + int(list[i-1])
        elif "triệu" in list[i]:
            total = total + int(list[i-1])/1000
    return total, raw

# Sửa parse_area_m2 để hiểu cả 'm 2', 'm^2', 'm²'
def parse_area_m2(text):
    if not text: return None
    t = clean_text(text).lower()
    # Chuẩn hoá các biến thể 'm 2', 'm ^ 2', 'm²' -> 'm2'
    t = t.replace("m²", "m2")
    t = re.sub(r"m\s*\^\s*2", "m2", t)
    t = re.sub(r"m\s*2", "m2", t)
    m = re.search(r"([\d\.,]+)\s*m2\b", t)
    if not m: return None
    s = m.group(1).replace(",", ".")
    s = re.sub(r"[^0-9.]", "", s)
    try:
        return float(s)
    except:
        return None

def first_int(text):
    if not text: return None
    m = re.search(r"\d+", text)
    return int(m.group()) if m else None

def extract_info_attr_rows(soup):
    """
    Đọc từng dòng thuộc tính trong div.info-attr.clearfix:
      <div class="info-attr clearfix">
        <span>Diện tích sử dụng</span>
        <span>78 m<sup>2</sup></span>
      </div>
    Trả về dict {label_lower: value_text}
    """
    info = {}
    # Tất cả dòng thuộc tính
    rows = soup.select("div.info-attr.clearfix")
    # Fallback nếu DOM khác một chút
    if not rows:
        container = soup.select_one("div.info-attrs.clearfix")
        if container:
            rows = container.select("div.info-attr")

    for row in rows:
        spans = row.find_all("span", recursive=False)  # ưu tiên span con trực tiếp
        if len(spans) >= 2:
            label = clean_text(spans[0].get_text(" ", strip=True)).lower()
            # Lấy value không chèn delimiter để 'm' + <sup>2> thành 'm2' (get_text("", strip=True))
            value = clean_text(spans[1].get_text("", strip=True))
            if label and value:
                info[label] = value
        else:
            # Trường hợp hiếm: label:value chung một span
            t = clean_text(row.get_text(" ", strip=True))
            if ":" in t:
                k, v = t.split(":", 1)
                k = clean_text(k).lower()
                v = clean_text(v)
                if k and v:
                    info[k] = v
    return info

def pick_value(pairs, keys):
    for lab, val in pairs.items():
        lab_l = lab.lower()
        if any(k in lab_l for k in keys):
            return val
    return None

def parse_detail_htmlparser(url):
    soup = get_soup(url)
    if not soup: return None

    # Tiêu đề
    h1 = soup.find("h1")
    tieu_de = clean_text(h1.get_text(" ")) if h1 else None

    # Giá raw từ div.price -> quy đổi bằng parse_price_ty_vnd (đơn vị tỷ)
    price_el = soup.find("div", class_="price")
    gia_text = clean_text(price_el.get_text(" ")) if price_el else None
    gia_vnd, gia_raw = parse_price_ty_vnd(gia_text)

    # Địa chỉ
    addr_el = soup.find("div", class_="address")
    dia_chi = clean_text(addr_el.get_text(" ")) if addr_el else None

    # Giới thiệu
    desc_el = soup.find("div", class_="info-content-body")
    gioi_thieu = clean_text(desc_el.get_text("\n")) if desc_el else None

    # Thuộc tính trong các dòng info-attr
    pairs = extract_info_attr_rows(soup)

    # Diện tích sử dụng / đất
    dt_sd_text  = pick_value(pairs, ["diện tích sử dụng", "dien tich su dung", "dtsd"])
    dt_dat_text = pick_value(pairs, ["diện tích đất", "dien tich dat", "dt đất", "dt dat"])
    if not dt_dat_text:
        # Fallback: nhiều tin chỉ có 'Diện tích'
        dt_dat_text = pick_value(pairs, ["diện tích", "dien tich"])

    dien_tich_su_dung_m2 = parse_area_m2(dt_sd_text) if dt_sd_text else None
    dien_tich_dat_m2     = parse_area_m2(dt_dat_text) if dt_dat_text else None

    # Phòng ngủ / nhà tắm (đã lấy được theo bạn, giữ nguyên cách đọc từ pairs)
    phong_ngu_text = pick_value(pairs, ["phòng ngủ","số phòng ngủ","so phong ngu","pn"])
    nha_tam_text   = pick_value(pairs, ["phòng tắm","nhà tắm","toilet","wc","số toilet","so toilet"])
    phong_ngu = first_int(phong_ngu_text)
    nha_tam   = first_int(nha_tam_text)

    return {
        "tieu_de": tieu_de,
        "link": url,
        "gia_raw": gia_raw,
        "gia_vnd": gia_vnd,
        "dia_chi": dia_chi,
        "dien_tich_dat_m2": dien_tich_dat_m2,
        "dien_tich_su_dung_m2": dien_tich_su_dung_m2,
        "phong_ngu": phong_ngu,
        "nha_tam": nha_tam,
        "phap_ly": pick_value(pairs, ["pháp lý","phap ly","giấy tờ","giay to","tinh trang phap ly"]),
        "gioi_thieu": gioi_thieu
    }

PATTERN_DETAIL = re.compile(r"-id(\d{5,})", re.I)  # nhận diện và trích ad_id

def extract_listing_links(listing_soup, base_url):
    links = []
    seen = set()
    if not listing_soup:
        return links
    for a in listing_soup.select("a[href]"):
        href = a.get("href","")
        if not href or href.startswith("#") or "javascript:" in href:
            continue
        full = urljoin(base_url, href).split("?")[0]
        m = PATTERN_DETAIL.search(full)
        if ("mogi.vn" in full) and m and full not in seen:
            seen.add(full)
            links.append(full)
    return links


def set_cp_param(url, page: int) -> str:
    parts = urlsplit(url)
    q = parse_qsl(parts.query, keep_blank_values=True)
    out = []
    seen_cp = False
    for k, v in q:
        if k.lower() == "cp":
            out.append(("cp", str(page)))
            seen_cp = True
        else:
            out.append((k, v))
    if not seen_cp:
        out.append(("cp", str(page)))
    new_query = urlencode(out, doseq=True)
    return urlunsplit((parts.scheme, parts.netloc, parts.path, new_query, parts.fragment))


def collect_links_by_cp(base_url, start_page=1, max_pages=30, sleep_range=(1.0, 2.0), break_no_new_pages=2):
    all_links = []
    seen_ids = set()
    no_new = 0

    for page in range(start_page, start_page + max_pages):
        page_url = set_cp_param(base_url, page)
        soup = get_soup(page_url)
        if not soup:
            print("Dừng do không tải được trang:", page_url)
            break

        links = extract_listing_links(soup, page_url)

        # Lọc link mới theo ad_id
        new_links = []
        for u in links:
            m = PATTERN_DETAIL.search(u)
            adid = m.group(1) if m else None
            if adid and adid not in seen_ids:
                seen_ids.add(adid)
                new_links.append(u)

        print(f"Trang cp={page}: {len(new_links)}/{len(links)} link mới; tổng {len(all_links)+len(new_links)}")

        if not new_links:
            no_new += 1
        else:
            no_new = 0

        all_links.extend(new_links)

        if no_new >= break_no_new_pages:
            print("Không thấy link mới trong nhiều trang liên tiếp → dừng.")
            break

        time.sleep(random.uniform(*sleep_range))

    return all_links

def ensure_csv(file_path, fieldnames):
    new = not os.path.exists(file_path)
    f = open(file_path, "a", newline="", encoding="utf-8-sig")
    w = csv.DictWriter(f, fieldnames=fieldnames)
    if new:
        w.writeheader()
    return f, w

def ad_id_from_url(u):
    m = PATTERN_DETAIL.search(u)
    return m.group(1) if m else None


def crawl_details_to_csv(detail_links, out_csv="./Data/mogi_dump.csv",
                         batch_size=100, sleep_range=(1.0, 2.0)):
    # các cột sẽ ghi
    cols = [
        "ad_id","tieu_de","link","dia_chi","gia_raw","gia_vnd",
        "dien_tich_dat_m2","dien_tich_su_dung_m2","phong_ngu","nha_tam","phap_ly","gioi_thieu"
    ]
    f, w = ensure_csv(out_csv, cols)
    written = 0
    batch = []

    try:
        for i, url in enumerate(detail_links, 1):
            try:
                item = parse_detail_htmlparser(url)  # dùng parser bạn đã xây
                if not item:
                    continue
                item["ad_id"] = ad_id_from_url(url)
                # chỉ giữ các cột cần
                row = {k: item.get(k) for k in cols}
                batch.append(row)
            except Exception as e:
                print("Lỗi parse:", url, e)

            # ghi theo lô
            if len(batch) >= batch_size:
                w.writerows(batch)
                written += len(batch)
                print(f"Đã ghi {written} bản ghi vào {out_csv}")
                batch.clear()

            time.sleep(random.uniform(*sleep_range))  # kiểm soát tốc độ

        # flush phần còn lại
        if batch:
            w.writerows(batch)
            written += len(batch)
            print(f"Đã ghi {written} bản ghi vào {out_csv}")
    finally:
        f.close()

## b.Scraping

In [24]:
#url = "https://mogi.vn/ho-chi-minh/quan-2/mua-nha"

#link_list = collect_links_by_cp(url, start_page=1, max_pages=1, sleep_range=(0.5, 1.5))

#crawl_details_to_csv(link_list)



## c.Data preprocessing

In [25]:
def norm_case(name):
    name = re.sub(r"\s+", " ", (name or "")).strip()
    return " ".join(w[:1].upper() + w[1:] for w in name.split())

def extract_quan_only(addr):
    """
    Trả về '1' hoặc 'Tan Phu' nếu tìm được; ngược lại trả None.
    Chỉ xử lý 'Quận', KHÔNG xử lý Huyện/TP.
    """
    text = addr or ""
    # Q.1 / Q1
    m = re.search(r"\bq\s*\.?\s*([0-9]{1,2})\b", text, flags=re.I)
    if m:
        return f"{int(m.group(1))}"
    # Quận 1
    m = re.search(r"(?:quận)\s*([0-9]{1,2})\b", text, flags=re.I)
    if m:
        return f"{int(m.group(1))}"
    # Quận Tân Phú
    m = re.search(r"(?:quận)\s*([^,;0-9]+)", text, flags=re.I)
    if m:
        return norm_case(m.group(1))
    return None

In [26]:
import re
import numpy as np
import pandas as pd

# 1) Đọc file CSV
path = "./Data/mogi_dump.csv"  # đổi nếu cần
df = pd.read_csv(path, encoding="utf-8-sig")

print("Loaded:", df.shape)
print("Columns:", df.columns.tolist())

# 2) Chọn và đổi tên cột
keep = {
    "dia_chi": "dia_chi",
    "phong_ngu": "phong_ngu",
    "nha_tam": "nha_tam",
    "dien_tich_dat_m2": "dien_tich_dat_m2",
    "dien_tich_su_dung_m2": "dien_tich_su_dung_m2",
    "gia_vnd": "gia" 
}
missing = [c for c in keep.keys() if c not in df.columns]
if missing:
    print("Thiếu cột trong file:", missing)

out = df[[c for c in keep.keys() if c in df.columns]].rename(columns=keep)

# 3) Chuẩn hoá giá trị trống -> NaN
out = out.replace(r'(?i)^\s*(na|n/a|none|null|nan)?\s*$', np.nan, regex=True)
out = out.replace(r'^\s*$', np.nan, regex=True)  # chuỗi rỗng/space

# 4) Ép kiểu số
num_cols = ["gia", "dien_tich_dat_m2", "dien_tich_su_dung_m2", "phong_ngu", "nha_tam"]
for c in num_cols:
    if c in out.columns:
        out[c] = pd.to_numeric(out[c], errors="coerce")

print("Số NaN theo cột trước khi drop:")
print(out.isna().sum())

# 5) Drop dòng nếu BẤT KỲ cột nào bị NaN
out_clean = out.dropna(how="any").copy().reset_index(drop=True)
print("Sau drop (any missing):", out_clean.shape)

out_clean["quan"] = out_clean["dia_chi"].apply(extract_quan_only)

# 7) Loại dòng không có 'quan' và bỏ cột 'dia_chi'
out_clean = out_clean.dropna(subset=["quan"]).drop(columns=["dia_chi"])

# 8) Sắp xếp lại cột và lưu file
cols_order = ["quan", "dien_tich_dat_m2", "dien_tich_su_dung_m2", "phong_ngu", "nha_tam","gia"]
out_clean = out_clean.reindex(columns=cols_order)

out_path = "./Data/data_predict_price_house.csv"
out_clean.to_csv(out_path, index=False, encoding="utf-8-sig")
print("Đã lưu:", out_path, "| shape:", out_clean.shape)
print(out_clean.head(5).to_string(index=False))

Loaded: (8532, 12)
Columns: ['ad_id', 'tieu_de', 'link', 'dia_chi', 'gia_raw', 'gia_vnd', 'dien_tich_dat_m2', 'dien_tich_su_dung_m2', 'phong_ngu', 'nha_tam', 'phap_ly', 'gioi_thieu']
Số NaN theo cột trước khi drop:
dia_chi                    0
phong_ngu                610
nha_tam                  613
dien_tich_dat_m2           0
dien_tich_su_dung_m2    4116
gia                        0
dtype: int64
Sau drop (any missing): (4294, 6)
Đã lưu: ./Data/data_predict_price_house.csv | shape: (4293, 6)
   quan  dien_tich_dat_m2  dien_tich_su_dung_m2  phong_ngu  nha_tam  gia
 Gò Vấp             147.0                 479.0       17.0     18.0 16.8
Tân Phú             180.0                 179.0       13.0     13.0 23.0
     10              53.0                  53.0        2.0      2.0  4.3
      9             233.0                 233.0       28.0     28.0 26.0
     10             102.0                 102.0        2.0      2.0  3.7


# d.Test Data:

In [27]:
import pandas as pd

out_path_test = "./Data/data_predict_price_house.csv"
data_test = pd.read_csv(out_path_test, encoding="utf-8-sig")

print("Test data shape:", data_test.shape)
print(data_test.head(5).to_string(index=False))

Test data shape: (4293, 6)
   quan  dien_tich_dat_m2  dien_tich_su_dung_m2  phong_ngu  nha_tam  gia
 Gò Vấp             147.0                 479.0       17.0     18.0 16.8
Tân Phú             180.0                 179.0       13.0     13.0 23.0
     10              53.0                  53.0        2.0      2.0  4.3
      9             233.0                 233.0       28.0     28.0 26.0
     10             102.0                 102.0        2.0      2.0  3.7


In [28]:
data_test.head()

,quan,dien_tich_dat_m2,dien_tich_su_dung_m2,phong_ngu,nha_tam,gia
0,Gò Vấp,147.0,479.0,17.0,18.0,16.8
1,Tân Phú,180.0,179.0,13.0,13.0,23.0
2,10,53.0,53.0,2.0,2.0,4.3
3,9,233.0,233.0,28.0,28.0,26.0
4,10,102.0,102.0,2.0,2.0,3.7
